In [1]:
'''
kaggle: kurtois
 add mean of ext 1,2,3
 add ratio of intersecting cols in (prev, train)
 drop some columns
 factorize some cols
 '''

'\nkaggle: kurtois\n drop some columns\n add mean of ext 1,2,3\n add ratio of intersecting cols in (prev, train)\n drop some columns\n '

In [2]:
import pandas as pd
import numpy as np
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
prev = pd.read_csv(PATH + 'previous_application.csv')
'done'

'done'

In [59]:
prev = pd.read_pickle(PATH + 'test/prev2curr_0.pkl')
print(prev.shape)
train = pd.read_pickle(PATH + 'inter/train_only_0.pkl')
test = pd.read_pickle(PATH + 'inter/test_only_0.pkl')

df = pd.concat([train, test], axis = 0)
df = df.merge(prev, how = 'left', on = 'SK_ID_CURR', sort = False)

(356255, 818)


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [47]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [15]:
# AMT_ANNUITY AMT_CREDIT AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START HOUR_APPR_PROCESS_START

In [52]:
amt_annuity = ['prev_AMT_ANNUITY_mean',
 'prev_AMT_ANNUITY_median',
 'prev_AMT_ANNUITY_max',
 'prev_AMT_ANNUITY_min',
 'prev_AMT_ANNUITY_sum']
amt_credit = ['prev_AMT_CREDIT_mean',
 'prev_AMT_CREDIT_median',
 'prev_AMT_CREDIT_max',
 'prev_AMT_CREDIT_min',
 'prev_AMT_CREDIT_sum']
amt_goods_price = ['prev_AMT_GOODS_PRICE_mean',
 'prev_AMT_GOODS_PRICE_median',
 'prev_AMT_GOODS_PRICE_max',
 'prev_AMT_GOODS_PRICE_min',
 'prev_AMT_GOODS_PRICE_sum']
week = ['prev_WEEKDAY_APPR_PROCESS_START_mean',
 'prev_WEEKDAY_APPR_PROCESS_START_median',
 'prev_WEEKDAY_APPR_PROCESS_START_max',
 'prev_WEEKDAY_APPR_PROCESS_START_min',
 'prev_WEEKDAY_APPR_PROCESS_START_sum']

list_df = ['AMT_ANNUITY','AMT_CREDIT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START']
list_prev = [amt_annuity, amt_credit, amt_goods_price, week]
for i in range(4):
    for j in range(4):
        print(ratio_name(list_df[i], list_prev[i][j]))
        df[ratio_name(list_df[i], list_prev[i][j])] = ratio(df, list_df[i], list_prev[i][j])


AMT_ANNUITY_divide_prev_AMT_ANNUITY_mean
AMT_ANNUITY_divide_prev_AMT_ANNUITY_median
AMT_ANNUITY_divide_prev_AMT_ANNUITY_max
AMT_ANNUITY_divide_prev_AMT_ANNUITY_min
AMT_CREDIT_divide_prev_AMT_CREDIT_mean
AMT_CREDIT_divide_prev_AMT_CREDIT_median
AMT_CREDIT_divide_prev_AMT_CREDIT_max
AMT_CREDIT_divide_prev_AMT_CREDIT_min
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_mean
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_median
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_max
AMT_GOODS_PRICE_divide_prev_AMT_GOODS_PRICE_min
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_mean
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_median
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_max
WEEKDAY_APPR_PROCESS_START_divide_prev_WEEKDAY_APPR_PROCESS_START_min


In [64]:

x = ['prev_AMT_APPLICATION_mean','prev_AMT_APPLICATION_max','prev_AMT_APPLICATION_min',\
     'prev_AMT_ANNUITY_mean','prev_AMT_ANNUITY_max','prev_AMT_ANNUITY_min',\
    'prev_AMT_CREDIT_mean','prev_AMT_CREDIT_max','prev_AMT_CREDIT_min']
for i in x:
    print(ratio_name('AMT_INCOME_TOTAL',i))
    df[ratio_name('AMT_INCOME_TOTAL',i)] = ratio(df, 'AMT_INCOME_TOTAL', i)

AMT_INCOME_TOTAL_divide_prev_AMT_APPLICATION_mean
AMT_INCOME_TOTAL_divide_prev_AMT_APPLICATION_max
AMT_INCOME_TOTAL_divide_prev_AMT_APPLICATION_min
AMT_INCOME_TOTAL_divide_prev_AMT_ANNUITY_mean
AMT_INCOME_TOTAL_divide_prev_AMT_ANNUITY_max
AMT_INCOME_TOTAL_divide_prev_AMT_ANNUITY_min
AMT_INCOME_TOTAL_divide_prev_AMT_CREDIT_mean
AMT_INCOME_TOTAL_divide_prev_AMT_CREDIT_max
AMT_INCOME_TOTAL_divide_prev_AMT_CREDIT_min


In [56]:
docs = [_f for _f in df.columns if 'FLAG_DOC' in _f]
print(docs)
df['NEW_DOC_IND_KURT'] = df[docs].kurtosis(axis=1) 
df.drop(docs,axis = 1, inplace = True)

In [ ]:
# Factorized: in train columns

In [ ]:
# Mean of exterior sec

In [61]:
ext_col = [col for col in df.columns if 'EXT' in col]
df['MEAN_EXT'] = df[ext_col].mean(axis = 1)

['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']